In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('../src'))
if module_path not in sys.path:
    sys.path.append(module_path)
import astropy.units as u


In [ ]:
import vsopy.util
import vsopy.data

OBJ_NAME='SZ Lyn'
SESSION_TAG='20250127'
IMAGE_ROOT = '/srv/public/img'
WORK_ROOT = '/srv/public'

session = vsopy.util.Session(tag=SESSION_TAG, name=OBJ_NAME)
layout = vsopy.util.WorkLayout(WORK_ROOT)
session_layout = layout.get_session(session)
IMAGE_DIR = vsopy.util.ImageLayout(IMAGE_ROOT).get_images(session).lights_dir
sd = vsopy.data.StarData(layout.charts_dir)

In [ ]:
import ccdproc as ccdp
import vsopy.reduce
import vsopy.phot
from astropy.table import QTable

N = 30
s = slice(N*3, (N+1)*3)
band = 'V'

matcher = vsopy.reduce.CalibrationMatcher(layout.calibr_dir)

stars = QTable.read(session_layout.root_dir / 'centroids.ecsv')

def reduce_image(path):
    solved = vsopy.reduce.load_and_solve(path, session_layout.solved_dir)
    cal = matcher.match(solved.header)
    reduced = vsopy.reduce.calibrate_image(solved,
                                dark=cal.dark,
                                flat=cal.flat)
    return reduced

# measure_stacked (IMAGE_DIR, V, slice)

def measure_stacked(dir, band, s):
    files = ccdp.ImageFileCollection(dir / band)
    paths = list(files.files_filtered(include_path=True))[s]
    reduced = list([reduce_image(path) for  path in paths])
    image = ccdp.combine(reduced, method='sum', mem_limit=1e9)
    image.header['EXPTIME'] = image.header['EXPTIME'] * len(reduced)

    phot_table = vsopy.phot.measure_photometry(image, stars,
                                    5*u.arcsec,
                                    (10*u.arcsec, 15*u.arcsec))
    return phot_table



In [ ]:
#images = list(files.ccds(ccd_kwargs={'unit': u.adu}))[0:3]
ph_v = measure_stacked(IMAGE_DIR, 'V', s)
ph_b = measure_stacked(IMAGE_DIR, 'B', s)

In [ ]:
ph_b

In [ ]:
import numpy as np

from astropy.table import join
chart = sd.get_chart(OBJ_NAME, fov=60*u.arcmin)

ph = join(join(ph_b['auid', 'M'], ph_v['auid', 'M'], keys='auid', table_names=['B', 'V']),
     chart['auid', 'B', 'V'], keys='auid', join_type='left')
ph.format = '.4f'

ph.rename_columns(['M_B', 'M_V'], ['instr B', 'instr V'])
ph['B']['mag'] = ph['B']['mag'].filled(np.nan)
ph['B']['err'] = ph['B']['err'].filled(np.nan)
ph['V']['mag'] = ph['V']['mag'].filled(np.nan)
ph['V']['err'] = ph['V']['err'].filled(np.nan)
ph['cat err'] = np.sqrt(ph['B']['err']**2 + ph['V']['err']**2)
ph['cat err'].format = '.4f'
ph['instr err'] = np.sqrt(ph['instr B']['err']**2 + ph['instr V']['err']**2)
ph['err'] = np.sqrt(ph['cat err']**2 + ph['instr err']**2)
ph

In [ ]:
target = '000-BBP-158'
comp = '000-BJR-416'
check = '000-BBP-198'
exclude = [target, comp, check, '000-BBP-105', '000-BBP-126']
tr = vsopy.phot.SimpleTransform.create(ph[~np.isin(ph['auid'], exclude)], 'B', 'V')
tr(ph[ph['auid'] == target], ph[ph['auid'] == comp]), tr(ph[ph['auid'] == check], ph[ph['auid'] == comp])

In [ ]:
tr1 = vsopy.phot.SimpleTransform.create(ph[~np.isin(ph['auid'], exclude)], 'V', 'B')
tr1(ph[ph['auid'] == target], ph[ph['auid'] == comp]), tr1(ph[ph['auid'] == check], ph[ph['auid'] == comp])

In [ ]:
import matplotlib.pyplot as plt

t = ph[~np.isin(ph['auid'], exclude)] #ph[~np.isin(ph['auid'], ['000-BBP-158', '000-BBP-105'])]

AB = t['B']['mag'] - t['V']['mag']
ab = t['instr B']['mag'] - t['instr V']['mag']
Aa = t['B']['mag'] - t[f'instr B']['mag']

sz = 2 * (16-t['B']['mag'].value) **2

fig = plt.figure(figsize=(10.24, 8.00))
gs = fig.add_gridspec(1, 1)
ax = plt.subplot()
ax = fig.add_subplot(gs[0, 0])
#ax.scatter(AB.value, ab.value, s=sz)
for x, y, label in zip(AB.value, ab.value, t['auid']):
    plt.text(x, y, label)
ax.errorbar(AB.value.unmasked, ab.value,
     #t['instr B']['mag'], t['B']['mag'],
             xerr=np.sqrt(t['B']['err'].unmasked.value**2 + t['V']['err'].unmasked.value**2)/2,
             yerr=np.sqrt(t['instr B']['err'].value**2 + t['instr V']['err'].value**2)/2,
             fmt='o')
# ax = fig.add_subplot(gs[0, 1])
# ax.scatter(AB, Aa)
# ax.errorbar(AB, Aa,
#     #t['instr V']['mag'], t['V']['mag'],
#             # xerr=t['instr B']['err']/2,
#             #yerr=np.ma.getdata(t['B']['err'])/2,
#             fmt='o')

plt.show()
#t['B']['err'])/2

In [ ]:
t['B']['err'].value

In [ ]:
from astropy.coordinates import SkyCoord

c = SkyCoord([r.wcs.wcs.crval for r in reduced], unit = u.deg)

[j.separation(i).to(u.arcsec)  for i, j in zip(c[:-1], c[1:])]